<a href="https://colab.research.google.com/github/sushi15/Twitter-Sentiment-Analysis/blob/main/Recalculate_sentiment_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import scipy.sparse

In [ ]:
tweetsM = scipy.sparse.load_npz('/content/gdrive/MyDrive/tweetsM.npz')

In [ ]:
tweetsM.toarray()

In [ ]:
dataframe = pd.read_csv('/content/gdrive/MyDrive/training.1600000.processed.noemoticon.csv',encoding ="ISO-8859-1",names=["target", "ids", "date", "flag", "user", "text"])
dataframe.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
len(dataframe)

1600000

In [ ]:
dataframe = dataframe.loc[dataframe['target'].isin([0,4])]

In [ ]:
len(dataframe)

1600000

In [ ]:
# dataframe = dataframe.drop(['ids', 'date', 'flag', 'user'], axis=1)
dataframe['target'] = [1 if target == 4 else target for target in dataframe['target']]

In [ ]:
dataframe["target"].value_counts()

0    800000
1    800000
Name: target, dtype: int64

In [ ]:
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english")

In [ ]:
stop_words[0:9] # Write to file 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you']

In [ ]:
#dataframe["text"] = dataframe["text"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
tweets = list(dataframe["text"]) 
label = list(dataframe["target"])

In [ ]:
tweets[0:9] # Write to file

["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D",
 "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!",
 '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds',
 'my whole body feels itchy and like its on fire ',
 "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. ",
 '@Kwesidei not the whole crew ',
 'Need a hug ',
 "@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?",
 "@Tatiana_K nope they didn't have it "]

In [ ]:
# Remove tags and urls 
urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
tagPattern       = '@[^\s]+'
# Also replacing three or more occurences of a letter with two occurences
repititionPattern   = r"(.)\1\1+"
repititionReplacePattern = r"\1\1"

def preprocess(tweets,stem=False):
  for i in range(len(tweets)): 
    tweet = tweets[i]
    tweet = tweet.lower()
    tweet = re.sub(urlPattern,'',tweet)
    tweet = re.sub(tagPattern,'', tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #Remove the #
    tweet = re.sub("[^a-zA-Z0-9]", " ", tweet)
    tweet = re.sub(repititionPattern,repititionReplacePattern, tweet)   
    words = []
    for word in tweet.split(): 
      if (word != " " and  word  not in stop_words):
        if stem:
          words.append(stemmer.stem(word))
        else:
          words.append(word)
    tweets[i] = " ".join(words)
  return tweets
   

### Stemming, remove URLs

In [ ]:
performStemming = False
tweet = preprocess(tweets,performStemming)

In [ ]:
tweets[0:9] # Write to file 

['aww bummer shoulda got david carr third day',
 'upset update facebook texting might cry result school today also blah',
 'dived many times ball managed save 50 rest go bounds',
 'whole body feels itchy like fire',
 'behaving mad see',
 'whole crew',
 'need hug',
 'hey long time see yes rains bit bit lol fine thanks',
 'nope']

In [ ]:
from textblob import TextBlob 

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
sa = SentimentIntensityAnalyzer()

In [ ]:
labels = [] 
for i in range(len(tweets)): 
  # tb = TextBlob(tweets[i]) 
  # sa = tb.sentiment 
  # if sa[0] < 0: 
  #   labels.append(0) 
  # else: 
  #   labels.append(1) 
  res = sa.polarity_scores(tweets[i]) 
  if res["compound"] < 0: 
    labels.append(0) 
  else: 
    labels.append(4)

In [ ]:
# tweets = list(dataframe["text"])
# for i in range(len(tweets)): 
#   print(tweets[i] + " : " + str(labels[i]))

Streaming output truncated to the last 5000 lines.
@lennel That is sooo sickening! Whya re people still living in the past man, such a shame  : 0
am in much pain from digging today. think i have to give up my lovely allotment. or swap to a smaller one. makes me cry thinking about it  : 0
worrying about so many things, and sleep deprived!!!!  : 0
@PapareBoy everything about vista is a bitch...  : 0
Mondayyyy  works okay though, my parents suck - couldn't find my shoes this morning  : 0
I've been on the phone since I got up and now I have to go to my sister's. Looks like no work today either.  : 4
wrapping web orders.  crap on TV.  finally run out of choc (easter eggs).  and wine  : 0
I want to sleep. Damn, why is so bad weather in Moscow? So cold  : 0
's family just collapsed. Oh dear time to be the super glue.... Again  : 4
@mileycyrus  so sad I can't go to the HM movie premiere in Paris next week  : 0
@Schofe Gutted im going to miss it, stuck in work  xxx : 0
Iam Sad I ForGot My PassW

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
i'm so confused on what to pack, how to pack it, and how to start. someone help  : 4
@KaraJonas How far is your school? For the next two weeks I'm going to have to walk to my bus stop which is about a mile away. FML for me  : 4
@Cosmetosaur I know!!   Darn MAC messing w/us again. The tiger bag seems out of place. Nothing from the collection is tiger printed haha. : 4
sad that i cant buy the Claudia wine in a glass  : 0
They think a tornado touched down in NC last night. Pray for the ones who have lost their homes, belongings and loved ones.  : 4
work....   : 4
@majm07 at least you're out of the house unlike me  : 4
@shaunjumpnow I wanted to go to groovin but none of my friends wanted to &amp; the tickets are sold out  I live in maitland &amp; really wanted to : 0
...sick and i hate it  last podcast of belgrade clubbing http://www.zshare.net/audio/59492816519ef9a0/ : 0
Hi ho, hi ho, it's off to pay the bills i go  : 0
@tamsul Yes..i'll 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
@timlovejoy you're quiet lately  : 4
Very grumpy this morning. I really hate wearing a suit sometimes.  : 0
@cherbie I just saw that I never gave U a heart, &amp; ythat broke my heart! I'm sorry!  : 0
@gr8conf Are the talks getting recorded for those of us unfortunate enough to miss the event?  #gr8conf : 0
has moved. Happy #Monday I have no net access after 5pm  sad day : 4
@tdh I think I'll be exploring the shit out outa' this wolfram, I am home sick! With nothing to do whole week !!  : 0
@MaraBG I received my invitation which says preapproved but at yahoo it won't let me in and says awaiting moderator approval.  : 4
good morning all. hope u have a better day than i do. sob  x : 4
can't concentrate ..   let's make a trip... : 4
good morn u guys...not so good for me  : 4
@jairodriguez sleeping alone does suck  : 0
Less joy now.  Sound stutters badly on my hackintosh and my &quot;bug-report&quot; on the forum just gets ignored.    Time

...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
@goaliemom31 THAT BLOWS!!!  More reason for the Wings to KILL Pitts...  Sorry for anyone you know there that is loosing a job  : 0
stupid allergies! i got finals tomorrow!!  : 0
Electricity is out  : 4
@sandeepparikh If you run out, there will be more right? I can't afford one right now  : 4
Where's my kitty cat when I need her?  : 4
@mimi_ignatieff I tried to erase Paul Martin from my memory. Just like Ray Simard.  : 4
@xoMusicLoverxo lesson learned, believe me! I was super upset  : 4
@Alow8111 If only it wasn't 30 mins away  : 4
i just had to clean up what seemed like a huge amount of blood because Jody's tooth started bleeding from being cleaned.   : 4
@ButterflyerGurl Meh I hate all of you  http://tinyurl.com/n8rbv8 : 0
Sleeping...sadly I have my history and spanish finals in the morning  : 0
I hope Rachael is okay  : 4
@cyborgturkey sadly it may not be  : 0
@ayechbee I can't seem to figure out to get an app for twitter on my bb  h

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
@fryfan20 What's wrong fryfan?  : 0
Ahhhh. I missed seeing JB on sportscenter.  &amp; Happy saturday!!!  Mariah (: : 4
@natleh_ yeah man! you trying to say i'm not?  : 4
In the forum, feelin very unhappy coz i didnt get the job at the cinema.  : 0
@AmandaMorante I miss my external HD...I can't re-assemble my portfolio without it   gotta wait til my sister brings it to Indiana... : 0
@StewartKris sorry someone hacked into your twitter  : 0
just gave a BIGGGGGGG goodbye hug to both manaum and areej  : 4
my knees and elbow hurt, i have the bones of a 75 year old!  : 0
Bwahaha, i am back home! Spent more than 100bucks today  Also not much sales items. Haha. : 4
@Diana_Vickers_ Morninggg diana  Its england what do we expect?  haa xxxx : 4
i have just returned from work and its raining  : 4
`jst woke up (bye) ate lunchz0r (mmm)` *cold, sad, scared*  http://plurk.com/p/ytt3d : 0
When I say &quot;silly&quot; I mean &quot;possibly autistic&quot

KeyboardInterrupt: ignored

In [ ]:
dataframe["target"] = labels 

In [ ]:
dataframe.to_csv("/content/gdrive/MyDrive/sentiment140Updated.csv", index = False, header = False, encoding ="ISO-8859-1")